In [4]:
import os
import json
import prompty
from coherence import coherence_evaluation
from relevance import relevance_evaluation
from fluency import fluency_evaluation
from groundedness import groundedness_evaluation

## Get output from data and save to results jsonl file

In [5]:
import pandas as pd

data_path = "../data/data.jsonl"

df = pd.read_json(data_path, lines=True)
df.head()

,customerId,question,chat_history,intent
0,4,tell me about your hiking jackets,[],chat
1,1,Do you have any climbing gear?,[],chat
2,3,Can you tell me about your selection of tents?,[],chat
3,6,Do you have any hiking boots?,[],chat
4,2,What gear do you recommend for hiking?,[],chat


In [ ]:

results = []

for index, row in df.iterrows():
    customerId = row['customerId']
    question = row['question']
    
    # Run contoso-chat/chat_request flow to get response
    response = get_response(customerId=customerId, question=question, chat_history=[])
    
    # Add results to list
    result = {
        'question': question,
        'context': response["context"],
        'answer': response["answer"]
    }
    results.append(result)

# Save results to a JSONL file
with open('result.jsonl', 'w') as file:
    for result in results:
        file.write(json.dumps(result) + '\n')

In [ ]:
# Evaluate results from results file
results_path = 'result.jsonl'
results = []
with open(results_path, 'r') as file:
    for line in file:
        results.append(json.loads(line))

for result in results:
    question = result['question']
    context = result['context']
    answer = result['answer']
    
    groundedness_score = groundedness_evaluation(question=question, answer=answer, context=context)
    fluency_score = fluency_evaluation(question=question, answer=answer, context=context)
    coherence_score = coherence_evaluation(question=question, answer=answer, context=context)
    relevance_score = relevance_evaluation(question=question, answer=answer, context=context)
    
    result['groundedness'] = groundedness_score
    result['fluency'] = fluency_score
    result['coherence'] = coherence_score
    result['relevance'] = relevance_score

# Save results to a JSONL file
with open('result_evaluated.jsonl', 'w') as file:
    for result in results:
        file.write(json.dumps(result) + '\n')

# Print results

df = pd.read_json('result_evaluated.jsonl', lines=True)
df.head()